# # Daily Extremes Timeseries from 2D Fields (25km IAF JRA55-do, ACCESS-OM3)

This notebook uses daily extreme values derived from 2D fields, using the [dev-MC_25km_jra_iaf configuration](https://github.com/ACCESS-NRI/access-om3-configs/tree/efc6d1f913b453898aa82fa40dbe133560c6bad2) of ACCESS-OM3.

In [5]:
#parameters

### USER EDIT start
esm_file='/g/data/tm70/ml0072/COMMON/git_repos/access-experiment-generator/2025.02.000/c1_25km_iaf_check-max-min-daily-only/perturb_1/MC_25km_jra_iaf/archive/experiment_datastore.json'
esm_file='/g/data/ol01/outputs/access-om3-25km/MC_25km_jra_iaf-1.0-beta-5165c0f8/datastore.json'
dpi=300
### USER EDIT stop

#CB note -- 19/2/26 - This notebook doesn't currently run on the latest runs as we do not have the relevant outputs, see https://github.com/ACCESS-NRI/access-om3-configs/issues/1046#issuecomment-3924389373

import os
from matplotlib import rcParams
%matplotlib inline
rcParams['figure.dpi']= dpi

plotfolder=f"/g/data/{os.environ['PROJECT']}/{os.environ['USER']}/access-om3-paper-1/notebooks/"
os.makedirs(plotfolder, exist_ok=True)

# needed for mkmd START 
notebook_name='not_using_mkfigs.sh'
notebooks_dir = os.path.join(f"/g/data/{os.environ['PROJECT']}/{os.environ['USER']}/access-om3-paper-1", "notebooks")
os.makedirs(notebooks_dir, exist_ok=True)
os.chdir(notebooks_dir)
print("Changed working directory to:", os.getcwd())
from mkfigs_configdoc import mkmd,get_notebook_name
notebook_name=get_notebook_name(notebook_name)
## example usage (note if the first argument to mkmd remains the same, you can insert multiple figures from the same notebook): 
#if notebook_name!='not_using_mkfigs.sh': plt.savefig(plotfolder+'mkmd/'+notebook_name[:-6]+'1.png', dpi=300, bbox_inches='tight')
#if notebook_name!='not_using_mkfigs.sh': mkmd("Template notebook","`"+notebook_name+"`: Example figure 1 of ACCESS-OM3 sea surface height (m).",os.path.basename(os.path.dirname(esm_file)),notebook_name[:-6]+'1.png',plotfolder+'mkmd/')
#if notebook_name!='not_using_mkfigs.sh': mkmd("Template notebook","`"+notebook_name+"`: Example figure 2 of ACCESS-OM3 sea surface height (m).",os.path.basename(os.path.dirname(esm_file)),notebook_name[:-6]+'2.png',plotfolder+'mkmd/')
# needed for mkmd STOP

 # a similar cell under this means it's being run in batch
print("ESM datastore path: ",esm_file)
print("Plot folder path: ",plotfolder)

Changed working directory to: /g/data/tm70/cyb561/access-om3-paper-1/notebooks
Notebook name is: not_using_mkfigs.sh
ESM datastore path:  /g/data/ol01/outputs/access-om3-25km/MC_25km_jra_iaf-1.0-beta-5165c0f8/datastore.json
Plot folder path:  /g/data/tm70/cyb561/access-om3-paper-1/notebooks/


In [2]:
import intake
from dask.distributed import Client
import matplotlib.pyplot as plt
import cftime

In [3]:
client = Client(threads_per_worker=1)
print(client.dashboard_link)

/proxy/8787/status


## Load ACCESS-OM3 data from ESM datastore

In [6]:
#datastore_path = "/g/data/tm70/ml0072/COMMON/git_repos/access-experiment-generator/2025.02.000/c1_25km_iaf_check-max-min-daily-only/perturb_1/MC_25km_jra_iaf/archive/experiment_datastore.json"

datastore = intake.open_esm_datastore(
    esm_file,
    columns_with_iterables=[
        "variable",
        "variable_long_name",
        "variable_standard_name",
        "variable_cell_methods",
        "variable_units"
    ]
)

In [ ]:
tos_min = datastore.search(variable="tos_min", frequency="1day").to_dask(
    xarray_combine_by_coords_kwargs = dict( # These kwargs can make things faster
        compat="override",
        data_vars="minimal",
        coords="minimal",
    ),
    xarray_open_kwargs = dict(
        chunks={"yh": -1, "xh": -1}, # Good for spatial operations, but not temporal
        decode_timedelta=True
    )
)

In [7]:
variables = ["tos_min", "tos_max", 
             "sos_min", "sos_max",
             "speed_min", "speed_max",
             "mlotst_min", "mlotst_max",
            ]
data = {
    v: datastore.search(variable=v, frequency="1day").to_dask(
        xarray_open_kwargs=dict(chunks={"time": -1}, decode_timedelta=True)
    )
    for v in variables
}

ValueError: Expected exactly one dataset. Received 0 datasets. Please refine your search on  or use `.to_dataset_dict()`.

In [ ]:
pairs = {}
for v in data:
    if v.endswith("_min"):
        pairs.setdefault(v[:-4], {})["min"] = v
    elif v.endswith("_max"):
        pairs.setdefault(v[:-4], {})["max"] = v

print(pairs)

fig, axes = plt.subplots(len(pairs), 2, figsize=(14, 3*len(pairs)), sharex=True)
if len(pairs) == 1:
    axes = axes[None, :]

for i, (base, pair) in enumerate(pairs.items()):
    if "min" not in pair or "max" not in pair:
        continue

    da_min = data[pair["min"]][pair["min"]]
    da_max = data[pair["max"]][pair["max"]]

    ts_min = da_min.min(dim=("yh", "xh"))
    ts_max = da_max.max(dim=("yh", "xh"))
    units  = da_min.attrs.get("units")

    ts_min.plot(ax=axes[i, 0], color="b", linewidth=1.5, label="Daily Min (spatial min)")
    axes[i, 0].set_title(f"{base} - Daily Min (spatial min)")
    axes[i, 0].set_ylabel(units); axes[i, 0].grid(True); axes[i, 0].legend()

    ts_max.plot(ax=axes[i, 1], color="r", linewidth=1.5, label="Daily Max (spatial max)")
    axes[i, 1].set_title(f"{base} - Daily Max (spatial max)")
    axes[i, 1].set_ylabel(units); axes[i, 1].grid(True); axes[i, 1].legend()

axes[-1, 0].set_xlabel("Time")
axes[-1, 1].set_xlabel("Time")
plt.tight_layout()

plt.savefig(f"{plotfolder}/timeseries_vars_minmax.png", dpi=150)


if notebook_name!='not_using_mkfigs.sh': plt.savefig(plotfolder+'mkmd/'+notebook_name[:-6]+'1.png', dpi=300, bbox_inches='tight')
if notebook_name!='not_using_mkfigs.sh': mkmd("Timeseries Extreme Values","`"+notebook_name+"`: Extreme value timeseries.",os.path.basename(os.path.dirname(esm_file)),notebook_name[:-6]+'1.png',plotfolder+'mkmd/')

plt.show()

In [ ]:
client.close()